# === SwarmUI Updater Setup ===
The following cells will download and run a script to update SwarmUI to its latest tag/branch.

In [ ]:
import os

# Define URL and output filename for the SwarmUI update script
SWARM_UPDATER_SCRIPT_URL = "https://raw.githubusercontent.com/remphan1618/Red/main/updatelinucks.sh"
SWARM_UPDATER_SCRIPT_NAME = "updatelinucks.sh"
WORKSPACE_DIR = "/workspace"
SWARM_UPDATER_SCRIPT_PATH = os.path.join(WORKSPACE_DIR, SWARM_UPDATER_SCRIPT_NAME)

print(f"Target script path: {SWARM_UPDATER_SCRIPT_PATH}")

# Ensure workspace directory exists (it usually does in notebook environments)
if not os.path.exists(WORKSPACE_DIR):
    os.makedirs(WORKSPACE_DIR)
    print(f"Created directory: {WORKSPACE_DIR}")

# Download the .sh script using curl or wget
print(f"Attempting to download {SWARM_UPDATER_SCRIPT_NAME} to {SWARM_UPDATER_SCRIPT_PATH}...")
download_command_curl = f"curl -sSL -o '{SWARM_UPDATER_SCRIPT_PATH}' '{SWARM_UPDATER_SCRIPT_URL}'"
download_command_wget = f"wget -q -O '{SWARM_UPDATER_SCRIPT_PATH}' '{SWARM_UPDATER_SCRIPT_URL}'"

import subprocess
try:
    subprocess.run(download_command_curl, shell=True, check=True, capture_output=True)
    print(f"✅ Downloaded using curl.")
except subprocess.CalledProcessError:
    print(f"Curl failed or not found, trying wget...")
    try:
        subprocess.run(download_command_wget, shell=True, check=True, capture_output=True)
        print(f"✅ Downloaded using wget.")
    except subprocess.CalledProcessError as e_wget:
        print(f"❌ wget also failed. Error: {e_wget.stderr.decode() if e_wget.stderr else 'Unknown wget error'}")
        print(f"Please ensure curl or wget is installed and the URL is correct.")
        raise

# Verify download
if os.path.exists(SWARM_UPDATER_SCRIPT_PATH) and os.path.getsize(SWARM_UPDATER_SCRIPT_PATH) > 0:
    print(f"✅ Script {SWARM_UPDATER_SCRIPT_NAME} downloaded successfully to {SWARM_UPDATER_SCRIPT_PATH}.")
else:
    print(f"❌ Script download failed or the file is empty at {SWARM_UPDATER_SCRIPT_PATH}.")
    raise FileNotFoundError(f"Failed to download or verify {SWARM_UPDATER_SCRIPT_PATH}")

# Make the script executable
print(f"Making {SWARM_UPDATER_SCRIPT_PATH} executable...")
chmod_command = f"chmod +x '{SWARM_UPDATER_SCRIPT_PATH}'"
try:
    subprocess.run(chmod_command, shell=True, check=True)
    print(f"✅ Execution permissions granted for {SWARM_UPDATER_SCRIPT_PATH}.")
except subprocess.CalledProcessError as e_chmod:
    print(f"❌ Failed to grant execution permissions. Error: {e_chmod.stderr.decode() if e_chmod.stderr else 'Unknown chmod error'}")
    raise

# Run the script to update SwarmUI
# Note: This script will run with the notebook's environment and permissions.
# It assumes 'updatelinucks.sh' handles its own cd into the SwarmUI directory if needed.
print(f"Running {SWARM_UPDATER_SCRIPT_PATH} to update SwarmUI...")
run_script_command = f"'{SWARM_UPDATER_SCRIPT_PATH}'"
try:
    # Running with shell=True. Ensure the script is safe.
    # Capture_output=False so we see the script's output directly in the notebook cell output.
    process = subprocess.Popen(run_script_command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, cwd=WORKSPACE_DIR)
    for line in process.stdout:
        print(line, end='')
    process.wait()
    if process.returncode == 0:
        print(f"\n✅ {SWARM_UPDATER_SCRIPT_NAME} executed successfully.")
    else:
        print(f"\n❌ {SWARM_UPDATER_SCRIPT_NAME} exited with error code {process.returncode}.")
        # If you want the notebook to fail hard here, uncomment the next line
        # raise subprocess.CalledProcessError(process.returncode, run_script_command)
except Exception as e_run:
    print(f"❌ An error occurred while trying to run the script: {e_run}")
    raise

# === Original ComfyUI Node Installer Starts Below ===
---

# ComfyUI Node Installer

This notebook installs ComfyUI nodes for SwarmUI integration. It will:
- Check for ComfyUI directory structure
- Install or update specified ComfyUI nodes
- Handle requirements installation for each node

## Configuration

Set up the ComfyUI directory path and list of nodes to install:

In [ ]:
import os
import subprocess
import sys
from pathlib import Path

# Set ComfyUI directory to match SwarmUI's structure
COMFYUI_DIR = "./dlbackend/ComfyUI"

# List of nodes to install
NODES = [
    "https://github.com/Comfy-Org/ComfyUI-Manager",
    "https://github.com/kijai/ComfyUI-KJNodes",
    "https://github.com/aria1th/ComfyUI-LogicUtils",
    "https://github.com/crystian/ComfyUI-Crystools",
    "https://github.com/Kosinkadink/ComfyUI-VideoHelperSuite",
    "https://github.com/rgthree/rgthree-comfy",
    "https://github.com/calcuis/gguf",
    "https://github.com/city96/ComfyUI-GGUF"
]

print(f"ComfyUI Directory: {COMFYUI_DIR}")
print(f"Number of nodes to install: {len(NODES)}")

## Directory Validation

Check if the ComfyUI directory exists:

In [ ]:
# Check if ComfyUI directory exists
comfyui_path = Path(COMFYUI_DIR)

if not comfyui_path.exists():
    print(f"❌ Error: ComfyUI directory not found at {COMFYUI_DIR}")
    print("Please run SwarmUI installation first or ensure the path is correct.")
    # Depending on the environment, you might want to create it or raise an error
    # For now, let's assume if it's not there, the node installer part might not make sense
    # raise FileNotFoundError(f"ComfyUI directory not found: {COMFYUI_DIR}")
else:
    print(f"✅ ComfyUI directory found at {COMFYUI_DIR}")
    
# Create custom_nodes directory if it doesn't exist
custom_nodes_dir = comfyui_path / "custom_nodes"
if comfyui_path.exists(): # Only try to create custom_nodes if ComfyUI base exists
    custom_nodes_dir.mkdir(exist_ok=True)
    print(f"✅ Custom nodes directory ready at {custom_nodes_dir}")
else:
    print(f"ℹ️ ComfyUI directory does not exist, so custom_nodes directory was not created.")

## Virtual Environment Setup

Check for and activate ComfyUI's virtual environment if it exists:

In [ ]:
# Check for virtual environment
if 'comfyui_path' in locals() and comfyui_path.exists():
    venv_path = comfyui_path / "venv" / "bin" / "activate"
else:
    venv_path = Path("non_existent_path_placeholder") # Avoid error if comfyui_path not defined

swarm_no_venv = os.environ.get("SWARM_NO_VENV")

if venv_path.exists() and not swarm_no_venv:
    print("✅ ComfyUI virtual environment found")
    print("Note: In Jupyter, direct venv activation via 'source' doesn't work as in a shell.")
    print("The Python kernel running this notebook should ideally be from this venv, or pip commands need to target its Python executable.")
else:
    print("ℹ️  No ComfyUI virtual environment found at the expected location, or SWARM_NO_VENV is set.")
    print("Using the Python environment of the current Jupyter kernel.")

## Node Installation Functions

Define helper functions for installing nodes:

In [ ]:
def run_shell_command(command_list, cwd=None, shell=False):
    """Run a shell command and return the result"""
    try:
        print(f"  Running command: {' '.join(command_list) if isinstance(command_list, list) else command_list}")
        result = subprocess.run(
            command_list, 
            cwd=cwd, 
            capture_output=True, 
            text=True, 
            check=True,
            shell=shell # Use shell=True if command is a string, False for a list of args
        )
        # print(f"  Stdout: {result.stdout}") # Can be verbose
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"❌ Command failed: {' '.join(e.cmd) if isinstance(e.cmd, list) else e.cmd}")
        print(f"  Return code: {e.returncode}")
        print(f"  Stdout: {e.stdout}")
        print(f"  Stderr: {e.stderr}")
        return None

def install_node_requirements(requirements_file_path):
    """Install requirements from a requirements.txt file"""
    if requirements_file_path.exists():
        print(f"  📦 Installing requirements from {requirements_file_path}")
        # Ensure we use the correct pip, especially if not in a venv for ComfyUI
        pip_executable = sys.executable.replace("python", "pip") # Basic assumption
        command = [pip_executable, "install", "--no-cache-dir", "-r", str(requirements_file_path)]
        result = run_shell_command(command)
        if result is not None:
            print("  ✅ Requirements installed successfully.")
            return True
        else:
            print("  ❌ Failed to install requirements.")
            return False
    return True # No requirements.txt found is not an error for this step

print("✅ Helper functions for node installation defined")

## Install/Update Nodes

Install or update each ComfyUI node (if ComfyUI directory exists):

In [ ]:
def install_comfy_nodes():
    """Install or update all ComfyUI nodes"""
    if 'comfyui_path' not in locals() or not comfyui_path.exists():
        print("ℹ️ ComfyUI directory does not exist. Skipping ComfyUI node installation.")
        return
        
    print("🚀 Installing/Updating ComfyUI nodes...\n")
    
    success_count = 0
    total_count = len(NODES)
    
    for i, repo_url in enumerate(NODES, 1):
        dir_name = repo_url.split("/")[-1].replace(".git", "") # Get repo name from URL
        node_path = custom_nodes_dir / dir_name
        requirements_file = node_path / "requirements.txt"
        
        print(f"[{i}/{total_count}] Processing: {dir_name} ({repo_url})")
        
        if node_path.exists() and (node_path / ".git").is_dir():
            print(f"  📁 Node '{dir_name}' already exists - Attempting update...")
            if run_shell_command(["git", "pull"], cwd=node_path) is not None:
                print(f"  ✅ Updated '{dir_name}' successfully.")
                if install_node_requirements(requirements_file):
                    success_count += 1
            else:
                print(f"  ❌ Failed to update '{dir_name}'. Check git output above.")
        elif node_path.exists() and not (node_path / ".git").is_dir():
            print(f"  ⚠️ Directory '{dir_name}' exists but is not a git repository. Skipping.")
            print(f"     Please remove or backup '{node_path}' and re-run if you want to clone it fresh.")
        else:
            print(f"  📥 Installing new node '{dir_name}'...")
            # Clone new repository
            command = ["git", "clone", repo_url, str(node_path), "--recursive"]
            if run_shell_command(command) is not None:
                print(f"  ✅ Cloned '{dir_name}' successfully.")
                if install_node_requirements(requirements_file):
                    success_count += 1
            else:
                print(f"  ❌ Failed to clone '{dir_name}'. Check git output above.")
        
        print()  # Empty line for readability
    
    print(f"🎉 ComfyUI Node installation/update process complete!")
    print(f"✅ Successfully processed: {success_count}/{total_count} nodes that were either new or updated.")
    
    if success_count < total_count:
        print(f"⚠️  {total_count - success_count} nodes might have had issues or were skipped - check the output above.")

# Run the ComfyUI node installation
install_comfy_nodes()

## Verification (ComfyUI Nodes)

Verify that ComfyUI nodes were installed correctly (if ComfyUI directory exists):

In [ ]:
def verify_comfy_nodes():
    if 'comfyui_path' not in locals() or not comfyui_path.exists():
        print("ℹ️ ComfyUI directory does not exist. Skipping ComfyUI node verification.")
        return
        
    print("📋 Installed ComfyUI Nodes Verification:")
    print("=" * 40)

    installed_node_dirs = []
    for repo_url in NODES:
        dir_name = repo_url.split("/")[-1].replace(".git", "")
        node_path = custom_nodes_dir / dir_name
        
        status = "❌ Not found"
        if node_path.exists():
            if (node_path / ".git").is_dir():
                status = "✅ Installed (git repo)"
                installed_node_dirs.append(dir_name)
            else:
                status = "⚠️  Directory exists (not a git repo)"
        
        print(f"{status} - {dir_name}")

    print(f"\n📊 Summary: {len(installed_node_dirs)}/{len(NODES)} expected ComfyUI nodes are present as git repositories.")

# Verify ComfyUI nodes
verify_comfy_nodes()

## Next Steps

After running this notebook:

1. **Restart ComfyUI**: If ComfyUI was running, restart it to load the new nodes
2. **Check Node Manager**: Use ComfyUI-Manager to verify nodes are loaded correctly
3. **Test Functionality**: Try using the new nodes in your ComfyUI workflows
4. **Update Regularly**: Re-run this notebook periodically to update nodes and SwarmUI

## Troubleshooting

If you encounter issues:
- Ensure you have git, curl, and wget installed and accessible in the notebook environment.
- Check that you have proper permissions to write to the `/workspace/` and ComfyUI directories.
- Verify your internet connection for downloading repositories and scripts.
- Check the ComfyUI logs for any node loading errors if nodes don't appear after installation.